In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# initalizing variables
data_dir = '../data/trunk12'
nii_dir = '../data/trunk12_nii'
batch_size = 32
split_ratio = 0.9

# specify transformation functions to apply on each image
transform = transforms.Compose([transforms.ToTensor()])

# read images from the dataset directory
dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)
total_num = len(dataset.targets)
train_num = int(split_ratio * total_num)
val_num = total_num - train_num

plt.figure(figsize=[15, 5])
p = sns.countplot(dataset.targets, palette=['#2F3C7E', '#CCCCCC'])
p.set_xticklabels(dataset.classes);

In [ ]:
# split the images into train and test set
train_dataset, test_dataset = torch.utils.data.random_split(dataset,[train_num, val_num])

# generate a dataloader for the train set
train_loader = torch.utils.data.DataLoader(train_dataset.dataset,
                                            batch_size=batch_size,
                                            shuffle=True)

# generate a dataloader for the test set
test_loader = torch.utils.data.DataLoader(test_dataset.dataset,
                                            batch_size=batch_size,
                                            shuffle=True)

In [ ]:
# show multiple examples of images from the training set
train_batch = next(iter(train_loader))

n = 8
plt.figure(figsize=[15,5])
for i in range(n):
    img = train_batch[0][i]
    img = torch.permute(img, (1,2,0))
    target = dataset.classes[train_batch[1][i].item()]
    plt.subplot(1,n,i+1)
    plt.imshow(img)
    plt.title(target)
    plt.axis('off')
plt.show()


In [ ]:
import numpy as np
import nibabel as nib
from PIL import Image
import os

imgs = dataset.imgs

for file, label in imgs:
    img = Image.open(file)
    w2 = int(img.size[0]/2)
    h2 = int(img.size[1]/2)
    s = 128
    img = img.crop((w2-s, h2-s, w2+s, h2+s))
    arr = np.asarray(img)
    empty_header = nib.Nifti1Header()
    affine =  np.array([[1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 1, 0],
                        [0, 0, 0, 1]])
    another_img = nib.Nifti1Image(arr, affine, empty_header)
    #print(another_img.header.get_data_shape())
    file = file.replace(data_dir, nii_dir)
    file = file.replace('.JPG', '.nii.gz')
    path = file.replace(file.split('/')[-1], "")
    os.makedirs(path, exist_ok = True)
    nib.save(another_img, file)

In [ ]:
import nibabel as nib

img = nib.load(dataset.imgs[0][0].replace(data_dir, nii_dir).replace('.JPG', '.nii.gz'))
img.shape

In [ ]:
#!pip install pyradiomics

In [ ]:
import numpy as np
from PIL import Image

# save a mask file
mask = np.ones(img.shape) *255
mask[:1, :1, :] = 0
mask = mask.astype(np.uint8)
mask_name = "mask.nii.gz"
print(np.unique(np.asarray(mask)))

empty_header = nib.Nifti1Header()
affine =  np.array([[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])
another_img = nib.Nifti1Image(mask, affine, empty_header)
print(another_img.header.get_data_shape())
nib.save(another_img, mask_name)

In [ ]:
import csv
import numpy as np

# write a csv file with location and label of each image in the train set
pyradiomics_header = ('Image','Mask', 'Label')
m_arr = [mask_name] * len(train_dataset.dataset.imgs)
img_label = train_dataset.dataset.imgs.copy()
rows = [(il[0].replace(data_dir, nii_dir).replace('.JPG', '.nii.gz'), m, 255) for m, il in zip(m_arr, img_label)]
rows.insert(0, pyradiomics_header)
arr = np.asarray(rows)
np.savetxt('pyradiomics_samples.csv', arr, fmt="%s", delimiter=",")



In [ ]:
import radiomics
from radiomics import featureextractor 

print(train_dataset.dataset.imgs[0])
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor()
output = extractor.execute(train_dataset.dataset.imgs[0][0].replace(data_dir, nii_dir).replace('.JPG', '.nii.gz'), mask_name, label=255)


In [ ]:
import six 
# Make an array of the values
features = np.array([])

for key, value in six.iteritems(output):
    if key.startswith("original_"):
        features = np.append ( features, output[key])

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(3,1,1)
plt.plot(features)
plt.yscale('log')
plt.title ( "Features from image");

In [ ]:
# Run Pyradiomics on pyradiomics_sample.csv, output to pyradi.csv
!pyradiomics -o pyradi_features.csv -f csv pyradiomics_samples.csv

In [ ]:
import pandas as pd

# Declare csv Filename from Pyradiomics (zscore scaled and merged)
fname = "pyradi_features.csv"

# Load data
pyradi_data = pd.read_csv(fname)
pyradi_data.shape

In [ ]:
pyradi_original = pyradi_data.iloc[:,25:]
pyradi_original.head()

In [ ]:
from scipy.stats import zscore

pyradi_original.apply(zscore)
pyradi_original.dropna(axis=1, how='all')
pyradi_original.shape



In [ ]:
pyradi_original.head()

In [ ]:
pyradi_original['target'] = train_dataset.dataset.targets
pyradi_original.shape

In [ ]:
pyradi_original.head()

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = pyradi_original.corr()
sns.heatmap(cor, annot=False, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["target"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.2]
relevant_features

In [ ]:
variables = relevant_features.axes[0].tolist()

pyradi_relevant = pyradi_original[variables]
pyradi_relevant.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold 
from sklearn.model_selection import cross_val_score

# Define Random Forest model
def get_RF_models():
	models = dict()
	i=0.8
	key = 'RF'
	models[key] = RandomForestClassifier(max_samples=i, n_estimators=30)
	return models

# Define Support Vector Machine model
def get_SVM_models():
	models = dict()
	i=0.8
	key = 'SVM'
	models[key] = SVC(kernel='rbf',probability=True)
	return models

def evaluate_models(model, X, y):
	# define the evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1)
	# evaluate the model and collect the results
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores

In [ ]:
import numpy as np

# prep classification column
y = pyradi_relevant['target']

## Run RF Classifier
RF_max_score = 0
for i in range(1,220):
  j = i+2
  if j > 215:
    j = 215
  X = relevant_features.iloc[:,2:j]
  pyradi_featurename = list(relevant_features.columns)[2:j]
  models = get_RF_models()
  # evaluate the models and store results
  for name, model in models.items():
    # evaluate the model
    scores = evaluate_models(model, X, y)
	  # store the results
    #m_scores.append(mean(scores))
    m_scores = np.mean(scores)
    if m_scores > RF_max_score:

      RF_max_score = m_scores
      RF_max_j = j-2
      RF_max_std = np.std(scores)
	  
	  # summarize the performance along the way
    #print('>%s %s %.3f %.3f (%.3f)' % (name, j-2, RF_max_score, m_scores, std(scores)))
    print('Processing n Features: %s' % (j-2))
print('>Model:RF  Maximum Score:%.3f  StdDev:(%.3f)  No. Features Used:%s' % (RF_max_score, RF_max_std, RF_max_j))